<a href="https://colab.research.google.com/github/ASAzimy/SamimProject/blob/main/FirstImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**🎓 Thesis Deep Learning Implementation: MSFT + CapsNet + Neural ODEs**

**✅ Step-by-Step Instructions with Code (Google Colab)**

**📁 Step 1: Upload and Load MSFT Dataset**

In [1]:
# Upload the file to Colab
from google.colab import files
uploaded = files.upload()


Saving MSFT_1986_2025-06-30.csv to MSFT_1986_2025-06-30.csv


In [12]:
# ✅ STEP 1: Install required libraries
!pip install ta torchdiffeq --quiet

# ✅ STEP 2: Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchdiffeq import odeint
from ta.momentum import ChandeMomentumOscillator
from ta.momentum import StochasticOscillator


ImportError: cannot import name 'ChandeMomentumOscillator' from 'ta.momentum' (/usr/local/lib/python3.11/dist-packages/ta/momentum.py)

In [14]:
# ✅ THESIS IMPLEMENTATION: CapsNet and Neural ODE on Stock Data (Enhanced with Hyperparameter Tuning)

# STEP 1: Install required libraries
!pip install ta torchdiffeq optuna --quiet

# STEP 2: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchdiffeq import odeint
from ta.momentum import ChandeMomentumOscillator, StochasticOscillator
import optuna

# STEP 3: Upload dataset
from google.colab import files
uploaded = files.upload()

# STEP 4: Load and clean dataset
file_path = list(uploaded.keys())[0]
df = pd.read_csv(file_path)
df = df.iloc[2:].copy()
df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
df['Date'] = pd.to_datetime(df['Date'])
numeric_cols = ['Close', 'High', 'Low', 'Open', 'Volume']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# STEP 5: Add technical indicators
df['CMO'] = ChandeMomentumOscillator(close=df['Close'], window=14).cmo()
stoch = StochasticOscillator(high=df['High'], low=df['Low'], close=df['Close'], window=10, smooth_window=3)
df['STC'] = stoch.stoch()
df.dropna(inplace=True)

# STEP 6: Normalize and sequence data
features = ['Close', 'High', 'Low', 'Open', 'Volume', 'CMO', 'STC']
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features])
sequence_length = 30
X, y = [], []
for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i])
    y.append(scaled_data[i, 0])
X = np.array(X)
y = np.array(y)

# Split into train and validation
split_idx = int(len(X) * 0.8)
X_train, X_val = X[:split_idx], X[split_idx:]
y_train, y_val = y[:split_idx], y[split_idx:]

# PyTorch dataset
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(TimeSeriesDataset(X_train, y_train), batch_size=64, shuffle=True)
val_loader = DataLoader(TimeSeriesDataset(X_val, y_val), batch_size=64, shuffle=False)

# CapsNet definition
class CapsNet1D(nn.Module):
    def __init__(self, capsule_dim=16, hidden_dim=128):
        super(CapsNet1D, self).__init__()
        self.conv1 = nn.Conv1d(7, 64, kernel_size=3, padding=1)
        self.primary_capsules = nn.Conv1d(64, 8 * capsule_dim, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(8 * capsule_dim * sequence_length, hidden_dim)
        self.out = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = F.relu(self.conv1(x))
        x = F.relu(self.primary_capsules(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.out(x).squeeze()

# Neural ODE architecture
class ODEFunc(nn.Module):
    def __init__(self, hidden_dim):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(7, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 7)
        )

    def forward(self, t, x):
        return self.net(x)

class ODEBlock(nn.Module):
    def __init__(self, odefunc):
        super(ODEBlock, self).__init__()
        self.odefunc = odefunc

    def forward(self, x):
        t = torch.tensor([0, 1]).float()
        return odeint(self.odefunc, x, t)[-1]

class ODEModel(nn.Module):
    def __init__(self, hidden_dim):
        super(ODEModel, self).__init__()
        self.odeblock = ODEBlock(ODEFunc(hidden_dim))
        self.fc = nn.Linear(7, 1)

    def forward(self, x):
        x = x[:, -1, :]
        x = self.odeblock(x)
        return self.fc(x).squeeze()

# CapsNet Optuna tuning
def objective_caps(trial):
    capsule_dim = trial.suggest_int('capsule_dim', 8, 32)
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)

    model = CapsNet1D(capsule_dim=capsule_dim, hidden_dim=hidden_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    model.train()
    for epoch in range(5):
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            preds = model(batch_x)
            loss = criterion(preds, batch_y)
            loss.backward()
            optimizer.step()

    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            pred = model(batch_x)
            preds.extend(pred.numpy())
            targets.extend(batch_y.numpy())
    return mean_squared_error(targets, preds, squared=False)

study = optuna.create_study(direction='minimize')
study.optimize(objective_caps, n_trials=10)
print("Best CapsNet params:", study.best_params)

best_caps_model = CapsNet1D(
    capsule_dim=study.best_params['capsule_dim'],
    hidden_dim=study.best_params['hidden_dim']
)
caps_optimizer = torch.optim.Adam(best_caps_model.parameters(), lr=study.best_params['lr'])
criterion = nn.MSELoss()

for epoch in range(10):
    for batch_x, batch_y in train_loader:
        caps_optimizer.zero_grad()
        output = best_caps_model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        caps_optimizer.step()
    print(f"[CapsNet] Epoch {epoch+1}: Loss = {loss.item():.4f}")

best_caps_model.eval()
caps_preds, true_vals = [], []
with torch.no_grad():
    for batch_x, batch_y in val_loader:
        preds = best_caps_model(batch_x)
        caps_preds.extend(preds.numpy())
        true_vals.extend(batch_y.numpy())
mae_caps = mean_absolute_error(true_vals, caps_preds)
rmse_caps = mean_squared_error(true_vals, caps_preds, squared=False)
print(f"CapsNet MAE: {mae_caps:.4f}, RMSE: {rmse_caps:.4f}")

# -------------------
# HYPERPARAMETER TUNING FOR NEURAL ODE
# -------------------
def objective_ode(trial):
    hidden_dim = trial.suggest_int("hidden_dim", 16, 128)
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)

    model = ODEModel(hidden_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    model.train()
    for epoch in range(5):
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            preds = model(batch_x)
            loss = criterion(preds, batch_y)
            loss.backward()
            optimizer.step()

    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            pred = model(batch_x)
            preds.extend(pred.numpy())
            targets.extend(batch_y.numpy())

    rmse = mean_squared_error(targets, preds, squared=False)
    return rmse

ode_study = optuna.create_study(direction="minimize")
ode_study.optimize(objective_ode, n_trials=10)
print("Best ODE params:", node_study.best_params)

best_ode_model = ODEModel(hidden_dim=node_study.best_params["hidden_dim"])
ode_optimizer = torch.optim.Adam(best_ode_model.parameters(), lr=node_study.best_params["lr"])
criterion = nn.MSELoss()

for epoch in range(10):
    for batch_x, batch_y in train_loader:
        node_optimizer.zero_grad()
        output = best_ode_model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        node_optimizer.step()
    print(f"[ODE] Epoch {epoch+1}: Loss = {loss.item():.4f}")

best_ode_model.eval()
ode_preds, true_vals_ode = [], []
with torch.no_grad():
    for batch_x, batch_y in val_loader:
        preds = best_ode_model(batch_x)
        node_preds.extend(preds.numpy())
        true_vals_ode.extend(batch_y.numpy())
mae_ode = mean_absolute_error(true_vals_ode, node_preds)
rmse_ode = mean_squared_error(true_vals_ode, node_preds, squared=False)
print(f"ODE MAE: {mae_ode:.4f}, RMSE: {rmse_ode:.4f}")

# PLOT RESULTS COMPARISON
plt.figure(figsize=(14, 5))
plt.plot(true_vals[:300], label='Actual')
plt.plot(caps_preds[:300], label='CapsNet')
plt.plot(node_preds[:300], label='Neural ODE')
plt.title("CapsNet vs Neural ODE Predictions")
plt.xlabel("Samples")
plt.ylabel("Normalized Close")
plt.legend()
plt.grid(True)
plt.show()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 15.3 MB/s eta 0:00:00


ImportError: cannot import name 'ChandeMomentumOscillator' from 'ta.momentum' (/usr/local/lib/python3.11/dist-packages/ta/momentum.py)